### Import important libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import pandas.util.testing as tm
import statistics 
import random

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

C:\Users\ioann\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

ImportError: cannot import name 'MultiOutputMixin'

### Loading .csv files into pandas dataFrame

In [ ]:
dev = pd.read_csv('devices.csv') # devices
notif = pd.read_csv('notifications.csv') # notifications

tr1 = pd.read_csv('transactions_1.csv') # Transactions (3 files)
tr2 = pd.read_csv('transactions_2.csv')
tr3 = pd.read_csv('transactions_3.csv')

us = pd.read_csv('users.csv') # Users

### Users dataframe

In [ ]:
us.info()

In [ ]:
us.describe()

In [ ]:
print("Variance of user_settings_crypto_unlocked is % s" %(statistics.variance(us.user_settings_crypto_unlocked)) )
print("Variance of attributes_notifications_marketing_push is % s" %(statistics.variance(us.attributes_notifications_marketing_push))) 
print("Variance of attributes_notifications_marketing_email is % s" %(statistics.variance(us.attributes_notifications_marketing_email))) 
print("Variance of num_contacts is % s" %(statistics.variance(us.num_contacts))) 
print("Variance of num_referrals is % s" %(statistics.variance(us.num_referrals))) 
print("Variance of num_successful_referrals is % s" %(statistics.variance(us.num_successful_referrals))) 

In [ ]:
print(us.shape)
print(len(np.unique(us['user_id'])))
print(us.isna().sum())
print(np.unique(us['plan']))
print(np.unique(us['plan']).sum)
#  print(us['plan'] == 'STANDARD')
us.sample(5)

In [ ]:
numerical = [column for column in us.columns if us[column].dtype != 'O']
print(numerical)

categorical = [column for column in us.columns if us[column].dtype == 'O']
print(categorical)

In [ ]:
msno.bar(us)

### Devices dataFrame

In [ ]:
print(dev.shape)
print(dev.isna().sum())
print(len(np.unique(dev['user_id'])))
print(np.unique(dev['brand']))
dev.sample(5)

### Notifications dataFrame

In [ ]:
print(notif.shape)
print(notif.isna().sum())
print(np.unique(notif['reason']))
print(np.unique(notif['status']))
print(np.unique(notif['channel']))
notif.sample(5)

In [ ]:
numerical = [column for column in notif.columns if notif[column].dtype != 'O']
print(numerical)

categorical = [column for column in notif.columns if notif[column].dtype == 'O']
print(categorical)

### Merge the two datasets "users" as us and "devices"  as dev into the new one called "ud"

In [ ]:
ud=pd.merge(us,dev,on='user_id')

In [ ]:
ud['age'] = 2020 - (ud[['birth_year']])
ud.head(2)

In [ ]:
#We can use the boxplot and the scatter plot in order to investigate how the age is distributed
fig, ax = plt.subplots(figsize=(10,4))
sns.boxplot(x = 'age', data = ud, orient = 'h', width = 0.9,fliersize = 7, showmeans=True, ax = ax)
plt.show()

In [ ]:
ud.age.hist()

In [ ]:
# Replace 'SILVER' and 'GOLD' with 'PAID' in order to have two categories of plan: 'STANDARD' and 'PAID'
ud['plan'] = ud['plan'].str.replace('SILVER', 'PAID')
ud['plan'] = ud['plan'].str.replace('GOLD', 'PAID')
print("Number of customers by plan ","\n" ,ud['plan'].value_counts())

In [ ]:
# Now make a dummie about plan, STANDARD=0 and PAID=1
plan_dict = {'STANDARD': 0, 'PAID': 1}
ud['plan'] = ud['plan'].map(plan_dict)
ud['plan'].value_counts()

In [ ]:
ud['brand'].value_counts()

In [ ]:
pd.crosstab(index=ud['brand'], columns=ud['plan'])

In [ ]:
#Only 32 obs of unknown, so we have to replace it somehow
# Replace 'Unknown' with 'Android' in order to have two big categories of brand: 'Android' and 'Apple'
ud['brand'] = ud['brand'].str.replace('Unknown', 'Android')
ud['brand'].value_counts()


In [ ]:
# Now make a dummie about devices, Android=0 and Apple=1
brand_dict = {'Android': 0, 'Apple': 1}
ud['brand'] = ud['brand'].map(brand_dict)
ud['brand'].value_counts()

In [ ]:
ud['attributes_notifications_marketing_push'].value_counts()

In [ ]:
pd.crosstab(index=ud['attributes_notifications_marketing_email'], columns=ud['attributes_notifications_marketing_push'])

In [ ]:
pd.crosstab(index=ud['attributes_notifications_marketing_email'], columns=ud['plan'])

In [ ]:
pd.crosstab(index=ud['attributes_notifications_marketing_push'], columns=ud['plan'])

In [ ]:
#random.sample([0, 1], 1)

In [ ]:
ud['attributes_notifications_marketing_push'].fillna(random.randint(0, 1),inplace=True)
#ud['attributes_notifications_marketing_push']
ud['attributes_notifications_marketing_email'].fillna(random.randint(0, 1),inplace=True)

In [ ]:
ud['attributes_notifications_marketing_push'].value_counts()

In [ ]:
ud['attributes_notifications_marketing_email'].value_counts()

In [ ]:
mask = np.zeros_like(ud.corr(), dtype=np.bool) 
mask[np.triu_indices_from(mask)] = False 
f, ax = plt.subplots(figsize=(8, 8))
plt.title('Pearson Correlation Matrix',fontsize=25)
sns.heatmap(ud.corr(),linewidths=0.25,vmax=1.0,square=True,cmap="GnBu", 
            linecolor='w',annot=True,mask=mask,cbar_kws={"shrink": .75});

In [ ]:
#ud['country'].value_counts()

In [ ]:
plt.figure(figsize=(3,3))
sns.heatmap(ud.groupby(['plan', 'user_settings_crypto_unlocked']).num_contacts.mean().unstack(),annot=True, fmt=".0f")

In [ ]:
plt.figure(figsize=(3,3))
sns.heatmap(ud.groupby([
        'plan', 'attributes_notifications_marketing_email']).num_contacts.mean().unstack(),annot=True, fmt=".0f")

In [ ]:
from datetime import datetime
d = datetime.now()
only_date, only_time = d.date(), d.time()
only_date

In [ ]:
import datetime as dt
ud['created_date'] = ud['created_date'].apply(pd.to_datetime)
ud['created_year']=pd.DatetimeIndex(ud['created_date']).year
ud['created_month']=pd.DatetimeIndex(ud['created_date']).month
#ud.info()

In [ ]:
ud['created_year'].value_counts()

In [ ]:
ud['created_month'].value_counts()

In [ ]:
ud_new=pd.concat([pd.get_dummies(ud['created_year']), ud], axis=1)

In [ ]:
ud_new.drop('created_year', axis=1, inplace=True)
ud_new.drop('num_referrals', axis=1, inplace=True)
ud_new.drop('num_successful_referrals', axis=1, inplace=True)
ud_new.drop('birth_year', axis=1, inplace=True)
ud_new.drop('created_date', axis=1, inplace=True)
ud_new.info()

In [ ]:
mask = np.zeros_like(ud_new.corr(), dtype=np.bool) 
mask[np.triu_indices_from(mask)] = False 
f, ax = plt.subplots(figsize=(8, 8))
plt.title('Pearson Correlation Matrix',fontsize=25)
sns.heatmap(ud_new.corr(),linewidths=0.25,vmax=1.0,square=True,cmap="GnBu", linecolor='w',annot=True,mask=mask,cbar_kws={"shrink": .75});

In [ ]:
ud_new.drop('user_id', axis=1, inplace=True)
ud_new.drop('country', axis=1, inplace=True)
ud_new.drop('city', axis=1, inplace=True)

In [ ]:
import copy
data=copy.deepcopy(ud_new)

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop(columns=['plan']).values
X_columns = data.drop(columns=['plan']).columns
y = data['plan'].values

In [ ]:
print(X.shape)
print(X_columns)
print(y.shape)
print(type(y))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
print("X train: ", X_train.shape, "\n",
      "y train: ", y_train.shape, "\n",
      "X test: ", X_test.shape, "\n",
      "y test: ", y_test.shape, "\n")

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier

var = VarianceThreshold()

In [ ]:
rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
rf.fit(X_train, y_train)
rf.feature_importances_

In [ ]:
for c, imp in sorted(zip(X_columns, rf.feature_importances_), key=lambda pair: pair[1], reverse=True):
  print('{:20}: {}'.format(c, imp))

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline